# Unclear:

- Is there a $2\pi$ factor in $\Gamma$?
- Which is the right distance from cloud to detector, 6.5 or 11 cm?

From "Laser_Cooling_Colorado.pdf"

$$
R = \frac{\pi \Gamma \left(\frac{I}{I_s}\right)}{1 + \left(\frac{I}{I_s}\right) + 4\left(\frac{\Delta}{\Gamma}\right)^2}
$$

where  
- \( R \) is the scattering rate (photons per second per atom),  
- \( $\Gamma = 6$ MHz \) is the natural linewidth,  
- \( $I$ \) is the total intensity, 6 MOT beams combined 
- \( $I_s = 4.1$ mW/cm² \) is the saturation intensity,  
- \( $\Delta = 10$ MHz \) is the detuning from resonance.

In [24]:
from scipy import constants as const
from uncertainties import ufloat
from uncertainties.umath import *
from scipy.constants import h, c, pi

In [ ]:
# Values:

lambda_laser = 780.24 * 1e-9 # error ???
responsivity = 0.45 # A/W for 780.246 nm laser (eye balled), maybe check data sheet, error ???
gain = 1.5e6 # V/A, how strongly current is amplified from the photodiode into an output voltage, error = +- 2 %
V_out =  0.51# measured after MOT is properly set up, error ???
vout_error = 0.01 # V
area_total = (4 * const.pi * 0.065**2) / 3 # 6.5 cm gap from fluorescence cloud to lense (lense goes to detector, detecor itself only 13 mm²)
area_detector = const.pi * (1.27*1e-2)**2 # area of lense which focuses all light onto detector

# measure voltage
V_offset = 31 # mV # measure when all light gets blocked / when dark and laser is off
v_offset_error = 1 # mV
V_noise = 0.0003 # V (noise) part of the error

In [26]:
# reform this equation V_out = gain * current_photons
current_photons = V_out / gain

# also current_photons = P_indicident_power (W) * responsivity (A/W)
P_indicident_power = current_photons / responsivity

# Incident Power to Photon Rate
E_photon = const.h * const.c / lambda_laser
N_photons = P_indicident_power / E_photon

print(f"Number of photons that arrive on detector: {N_photons:.2e}, or {(N_photons / const.N_A):.2e} mol")

Number of photons that arrive on detector: 2.97e+12, or 4.93e-12 mol


In [27]:
print(f"N_photons: {N_photons:.2e}")

area_ratio = area_total / area_detector

# calculate total amount of photons per second
N_photons_tot = area_ratio * N_photons
# in one calculation that would be:
N_should_be_the_same = lambda_laser * area_total * V_out / (const.h * const.c * gain * responsivity * area_detector)

print(f"Number of photons that arrive on detector: {N_photons:.2e}, or {(N_photons / const.N_A):.2e} mol")
print(f"Number of photons emitted from fluorescence cloud: {N_should_be_the_same:.2e}, or {(N_should_be_the_same / const.N_A):.2e} mol")
print(f"Number of photons emitted from fluorescence cloud: {N_photons_tot:.2e}, or {(N_photons_tot / const.N_A):.2e} mol")

N_photons: 2.97e+12
Number of photons that arrive on detector: 2.97e+12, or 4.93e-12 mol
Number of photons emitted from fluorescence cloud: 1.04e+14, or 1.72e-10 mol
Number of photons emitted from fluorescence cloud: 1.04e+14, or 1.72e-10 mol


In [28]:
from uncertainties import ufloat

# Define powers with uncertainty
beam_1_p = ufloat(2.89e-3, 0.05e-3)  # W
beam_2_p = ufloat(2.89e-3, 0.05e-3)  # W
beam_3_p = ufloat(2.89e-3, 0.05e-3)  # W

# Radius with uncertainty
radius = ufloat(0.42e-2, 0.1e-2)  # convert cm to m and add uncertainty

# Beam areas
beam_1_a = pi * radius**2
beam_2_a = pi * radius**2
beam_3_a = pi * radius**2

# Beam intensities
beam_1 = beam_1_p / beam_1_a
beam_2 = beam_2_p / beam_2_a
beam_3 = beam_3_p / beam_3_a

# Final intensity
I_E = 2 * (beam_1 + beam_2 + beam_3)

print(f"I = {I_E}")


I = (3.1+/-1.5)e+02


In [29]:
Gamma = 6 * 1e6 # 6 MHz
# I = 2 * (beam_1 + beam_2 + beam_3)# sum of 6 MOT beams
I_S = 4.1 * 1e-3 * 1e4 # 4.1 mW/cm²
Delta = 10 * 1e6 # 10 MHz

R = (Gamma * const.pi * (I_E.n) / I_S) / (1 + (I_E.n/I_S) + 4*(Delta / Gamma)**2 )


print(f"Scattering rate R = {R:.2e}")

# print(I, I_S)

Scattering rate R = 7.29e+06


In [30]:
# reorder equation of scattering rate R = photons / (s * atoms) to get number of atoms in fluorescent clour

# n_photons_tot [photons/s], R [photons/(s*atoms)]
N_atoms = N_photons_tot / R

print(f"Number of atoms in fluorescent cloud: {N_atoms:.2e} or {(N_atoms/const.N_A):.2e} mol")

Number of atoms in fluorescent cloud: 1.42e+07 or 2.36e-17 mol


In [31]:
# N = lambda_laser * area_total * V_out / (const.h * const.c * gain * responsivity * area_lense)
# print(N / R)

# (N_photons_tot) / (R)

In [32]:
from uncertainties import ufloat
from uncertainties.umath import *
from scipy.constants import h, c, pi

# Define quantities with uncertainties
lambda_laser_E = ufloat(lambda_laser, 0.1*1e-9)         # wavelength (m ± m)
area_total_E = ufloat(area_total, 0.84e-2)             # m² ± m²
V_out_E = ufloat(V_out, 0.01)                   # V ± V
gain_E = ufloat(gain, 30000)                    # unitless
responsivity_E = ufloat(responsivity, 0.01)            # A/W ± A/W
area_detector_E = ufloat(area_detector, 001e-5)             # m² ± m²

# for R:
I_S_E = ufloat(I_S, 1) # mW/cm²
Delta_E = ufloat(Delta, 1)
Gamma_E = ufloat(6e6, 0.05e6)  # 6 MHz ± 0.05 MHz

R = (Gamma_E * const.pi * (I_E / I_S_E) ) / (1 + (I_E/I_S_E) + 4*(Delta_E / Gamma_E)**2 )
N_photons = (lambda_laser_E * area_total_E * V_out_E / (h * c * gain_E * responsivity_E * area_detector_E))
N_atoms = N_photons / R

print("R", R)
print("N photons", N_photons)
print("N/R", N_atoms)
print(f"Nominal: {N_atoms.n:.3e}, Uncertainty: ±{N_atoms.s:.3e}")

print(f"{area_total_E:.4e}")


R (7.3+/-2.1)e+06
N photons (1.0+/-0.5)e+14
N/R (1.4+/-0.8)e+07
Nominal: 1.423e+07, Uncertainty: ±7.957e+06
(1.7698+/-0.8400)e-02


In [45]:
r_test = ufloat(0.1, 0.01)

area_total_test = (4 * const.pi * r_test**2) / 3

print(f"{area_total_E:.1e}")

area_total_E

(1.8+/-0.8)e-02


0.017697638615222503+/-0.0084